In [2]:
import os
import requests
from datetime import datetime, timedelta
import pandas as pd
import json

In [3]:
urlbase = "https://data.cityofnewyork.us/resource/"

In [4]:
def desde_fecha(fecha_str):
       return f'{fecha_str}T00:00:00.000'

In [5]:
def hasta_fecha(fecha_str):
   return f'{fecha_str}T23:59:59.000'

In [6]:
def extraccion_actual(ini, fin, token):
    url_eventos = f"{urlbase}tvpp-9vvx.json"
    
    param = {
        "$where": f"start_date_time >= '{ini}' AND start_date_time <= '{fin}'",    
    }

    header = {"X-App-Token": token}
    eventos = requests.get(url=url_eventos, params=param, headers=header)
    

    if eventos.status_code != 200:
        print(f"Error Parques: {eventos.text}")
 
    assert eventos.status_code == 200, "Error en la extracción de eventos"
    return eventos.json()




In [7]:
token = os.getenv('NYC_OPEN_DATA_TOKEN')
assert token is not None, "Falta la variable de entorno NYC_OPEN_DATA_TOKEN"


fecha_hoy_str = datetime.now().strftime('%Y-%m-%d')
fecha_actual = desde_fecha(fecha_hoy_str)
fecha_fin = hasta_fecha(fecha_hoy_str)


json_eventos = extraccion_actual(fecha_actual, fecha_fin, token)

In [8]:
df = pd.DataFrame(json_eventos)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 305 entries, 0 to 304
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   event_id             305 non-null    object
 1   event_name           305 non-null    object
 2   start_date_time      305 non-null    object
 3   end_date_time        305 non-null    object
 4   event_agency         305 non-null    object
 5   event_type           305 non-null    object
 6   event_borough        305 non-null    object
 7   event_location       305 non-null    object
 8   street_closure_type  305 non-null    object
 9   community_board      305 non-null    object
 10  police_precinct      305 non-null    object
 11  cemsid               294 non-null    object
 12  event_street_side    15 non-null     object
dtypes: object(13)
memory usage: 31.1+ KB


In [9]:

df['start_date_time'] = pd.to_datetime(df['start_date_time'], format='%Y-%m-%dT%H:%M:%S.%f', errors='coerce')
df["start_date_time"] = df["start_date_time"].dt.strftime('%H:%M:%S')
df['end_date_time'] = pd.to_datetime(df['end_date_time'], errors='coerce',  format='%Y-%m-%dT%H:%M:%S.%f')
df["end_date_time"] = df["end_date_time"].dt.strftime('%H:%M:%S')



In [10]:
df.columns
df = df.drop(["event_id", "event_agency", "street_closure_type", 'community_board','police_precinct', 'cemsid',
       'event_street_side' ], axis = 1)


In [16]:

riesgo_map = {
    'Parade': 10,
    'Athletic Race / Tour': 10,
    'Street Event': 8,
    'Special Event': 7,
    'Plaza Event': 6,
    'Plaza Partner Event': 6,
    'Theater Load in and Load Outs': 5,
    'Religious Event': 3,
    'Farmers Market': 2,
    'Sidewalk Sale': 2,
    'Production Event': 1,
    'Sport - Adult': 1,
    'Sport - Youth': 1,
    'Miscellaneous': 1,
    'Open Street Partner Event': 2
}


df['nivel_riesgo_tipo'] = df['event_type'].map(riesgo_map)

In [17]:
df

,event_name,start_date_time,end_date_time,event_type,event_borough,event_location,nivel_riesgo_tipo
0,Maintenance,00:00:00,23:59:00,Special Event,Manhattan,Fort Tryon Park: Billings Lawn,7
1,Cherry Lawn Closure,00:00:00,02:00:00,Special Event,Manhattan,Madison Square Park: Cherry Lawn,7
2,Veterans Lawn Closure,00:00:00,15:00:00,Special Event,Manhattan,Madison Square Park: Veterans Lawn,7
3,Dana Discovery Center Lawn,00:00:00,23:59:00,Special Event,Manhattan,Central Park: Dana Discovery Center Lawn,7
4,Lawn Closure - Diary Lawn,00:00:00,23:59:00,Special Event,Manhattan,Central Park: The Dairy,7
...,...,...,...,...,...,...,...
300,Soccer - Non Regulation,19:30:00,21:30:00,Sport - Youth,Brooklyn,Kaiser Park: Football-01,1
301,Soccer - Non Regulation,20:00:00,19:30:00,Sport - Youth,Brooklyn,Calvert Vaux Park: Soccer-01,1
302,Soccer - Non Regulation,20:00:00,22:00:00,Sport - Youth,Brooklyn,Calvert Vaux Park: Soccer-01,1
303,Soccer - Non Regulation,20:00:00,22:00:00,Sport - Youth,Brooklyn,Bush Terminal Park: Soccer-02,1
